In [24]:
corpus = [
    'Сатыбалды кеше неше дәптер сатып алды?',
    'Сатыбалды кеше неше кептер атып алды?',
    'Сатыбалды кеше көп дәптер сатып алды,',
    'Сатыбалды кеше көк кептер атып алды.',
    'Үш қыз да ұшқыш,',
    'Ұшқыш та үш қыз.',
    'Үш қыз – ұшқыш,',
    'Ұшқыш – үш қыз.',
    'Шын айтты ма?',
    'Сын айтты ма?',
    'Шын айтса да,',
    'Сын айтса да,',
    'Шыны – сын,',
    'Сыны – шын болсын!',
]

In [25]:
corpus = [x.lower() for x in corpus]

In [26]:
corpus

['сатыбалды кеше неше дәптер сатып алды?',
 'сатыбалды кеше неше кептер атып алды?',
 'сатыбалды кеше көп дәптер сатып алды,',
 'сатыбалды кеше көк кептер атып алды.',
 'үш қыз да ұшқыш,',
 'ұшқыш та үш қыз.',
 'үш қыз – ұшқыш,',
 'ұшқыш – үш қыз.',
 'шын айтты ма?',
 'сын айтты ма?',
 'шын айтса да,',
 'сын айтса да,',
 'шыны – сын,',
 'сыны – шын болсын!']

In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')

In [28]:
from collections import defaultdict

word_freqs = defaultdict(int)

for text in corpus:
    words_with_offsets = tokenizer.backend_tokenizer.pre_tokenizer.pre_tokenize_str(text)
    new_words = [word for word, offset in words_with_offsets]
    for word in new_words:
        word_freqs[word] += 1

print(word_freqs)

defaultdict(<class 'int'>, {'сатыбалды': 4, 'кеше': 4, 'неше': 2, 'дәптер': 2, 'сатып': 2, 'алды': 4, '?': 4, 'кептер': 2, 'атып': 2, 'көп': 1, ',': 6, 'көк': 1, '.': 3, 'үш': 4, 'қыз': 4, 'да': 3, 'ұшқыш': 4, 'та': 1, '–': 4, 'шын': 3, 'айтты': 2, 'ма': 2, 'сын': 3, 'айтса': 2, 'шыны': 1, 'сыны': 1, 'болсын': 1, '!': 1})


In [29]:
alphabet = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in alphabet:
            alphabet.append(letter)
alphabet.sort()

print(alphabet)

['!', ',', '.', '?', 'а', 'б', 'д', 'е', 'з', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'ш', 'ы', 'қ', 'ү', 'ұ', 'ә', 'ө', '–']


In [30]:
vocab = ["_"] + alphabet.copy()

In [31]:
vocab

['_',
 '!',
 ',',
 '.',
 '?',
 'а',
 'б',
 'д',
 'е',
 'з',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'ш',
 'ы',
 'қ',
 'ү',
 'ұ',
 'ә',
 'ө',
 '–']

In [32]:
splits = {word: [c for c in word] for word in word_freqs.keys()}

In [33]:
splits

{'сатыбалды': ['с', 'а', 'т', 'ы', 'б', 'а', 'л', 'д', 'ы'],
 'кеше': ['к', 'е', 'ш', 'е'],
 'неше': ['н', 'е', 'ш', 'е'],
 'дәптер': ['д', 'ә', 'п', 'т', 'е', 'р'],
 'сатып': ['с', 'а', 'т', 'ы', 'п'],
 'алды': ['а', 'л', 'д', 'ы'],
 '?': ['?'],
 'кептер': ['к', 'е', 'п', 'т', 'е', 'р'],
 'атып': ['а', 'т', 'ы', 'п'],
 'көп': ['к', 'ө', 'п'],
 ',': [','],
 'көк': ['к', 'ө', 'к'],
 '.': ['.'],
 'үш': ['ү', 'ш'],
 'қыз': ['қ', 'ы', 'з'],
 'да': ['д', 'а'],
 'ұшқыш': ['ұ', 'ш', 'қ', 'ы', 'ш'],
 'та': ['т', 'а'],
 '–': ['–'],
 'шын': ['ш', 'ы', 'н'],
 'айтты': ['а', 'й', 'т', 'т', 'ы'],
 'ма': ['м', 'а'],
 'сын': ['с', 'ы', 'н'],
 'айтса': ['а', 'й', 'т', 'с', 'а'],
 'шыны': ['ш', 'ы', 'н', 'ы'],
 'сыны': ['с', 'ы', 'н', 'ы'],
 'болсын': ['б', 'о', 'л', 'с', 'ы', 'н'],
 '!': ['!']}

In [34]:
def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

In [35]:
pair_freqs = compute_pair_freqs(splits)

In [36]:
pair_freqs

defaultdict(int,
            {('с', 'а'): 8,
             ('а', 'т'): 8,
             ('т', 'ы'): 10,
             ('ы', 'б'): 4,
             ('б', 'а'): 4,
             ('а', 'л'): 8,
             ('л', 'д'): 8,
             ('д', 'ы'): 8,
             ('к', 'е'): 6,
             ('е', 'ш'): 6,
             ('ш', 'е'): 6,
             ('н', 'е'): 2,
             ('д', 'ә'): 2,
             ('ә', 'п'): 2,
             ('п', 'т'): 4,
             ('т', 'е'): 4,
             ('е', 'р'): 4,
             ('ы', 'п'): 4,
             ('е', 'п'): 2,
             ('к', 'ө'): 2,
             ('ө', 'п'): 1,
             ('ө', 'к'): 1,
             ('ү', 'ш'): 4,
             ('қ', 'ы'): 8,
             ('ы', 'з'): 4,
             ('д', 'а'): 3,
             ('ұ', 'ш'): 4,
             ('ш', 'қ'): 4,
             ('ы', 'ш'): 4,
             ('т', 'а'): 1,
             ('ш', 'ы'): 4,
             ('ы', 'н'): 9,
             ('а', 'й'): 4,
             ('й', 'т'): 4,
             ('т', 'т'): 2,
  

In [37]:
sorted_pairs = sorted(pair_freqs.items(), key=lambda x: x[1], reverse=True)

# Iterate through the sorted pairs and print them out
for i, pair in enumerate(sorted_pairs):
    print(f"{i+1}. {pair[0]}: {pair[1]}")

1. ('т', 'ы'): 10
2. ('ы', 'н'): 9
3. ('с', 'а'): 8
4. ('а', 'т'): 8
5. ('а', 'л'): 8
6. ('л', 'д'): 8
7. ('д', 'ы'): 8
8. ('қ', 'ы'): 8
9. ('к', 'е'): 6
10. ('е', 'ш'): 6
11. ('ш', 'е'): 6
12. ('с', 'ы'): 5
13. ('ы', 'б'): 4
14. ('б', 'а'): 4
15. ('п', 'т'): 4
16. ('т', 'е'): 4
17. ('е', 'р'): 4
18. ('ы', 'п'): 4
19. ('ү', 'ш'): 4
20. ('ы', 'з'): 4
21. ('ұ', 'ш'): 4
22. ('ш', 'қ'): 4
23. ('ы', 'ш'): 4
24. ('ш', 'ы'): 4
25. ('а', 'й'): 4
26. ('й', 'т'): 4
27. ('д', 'а'): 3
28. ('н', 'е'): 2
29. ('д', 'ә'): 2
30. ('ә', 'п'): 2
31. ('е', 'п'): 2
32. ('к', 'ө'): 2
33. ('т', 'т'): 2
34. ('м', 'а'): 2
35. ('т', 'с'): 2
36. ('н', 'ы'): 2
37. ('ө', 'п'): 1
38. ('ө', 'к'): 1
39. ('т', 'а'): 1
40. ('б', 'о'): 1
41. ('о', 'л'): 1
42. ('л', 'с'): 1


In [38]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('т', 'ы') 10


In [39]:
merges = {('т', 'ы'): "ты"}
vocab.append("ты")

In [40]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

In [41]:
vocab

['_',
 '!',
 ',',
 '.',
 '?',
 'а',
 'б',
 'д',
 'е',
 'з',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'ш',
 'ы',
 'қ',
 'ү',
 'ұ',
 'ә',
 'ө',
 '–',
 'ты']

In [42]:
splits = merge_pair("т", "ы", splits)
print(splits["сатыбалды"])

['с', 'а', 'ты', 'б', 'а', 'л', 'д', 'ы']


In [59]:
vocab_size = 41

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

In [60]:
vocab

['_',
 '!',
 ',',
 '.',
 '?',
 'а',
 'б',
 'д',
 'е',
 'з',
 'й',
 'к',
 'л',
 'м',
 'н',
 'о',
 'п',
 'р',
 'с',
 'т',
 'ш',
 'ы',
 'қ',
 'ү',
 'ұ',
 'ә',
 'ө',
 '–',
 'ты',
 'ын',
 'са',
 'ал',
 'алд',
 'алды',
 'қы',
 'саты',
 'ке',
 'ше',
 'сын',
 'сатыб',
 'сатыбалды']

In [61]:
len(vocab)

41

In [62]:
splits

{'сатыбалды': ['сатыбалды'],
 'кеше': ['ке', 'ше'],
 'неше': ['н', 'е', 'ше'],
 'дәптер': ['д', 'ә', 'п', 'т', 'е', 'р'],
 'сатып': ['саты', 'п'],
 'алды': ['алды'],
 '?': ['?'],
 'кептер': ['ке', 'п', 'т', 'е', 'р'],
 'атып': ['а', 'ты', 'п'],
 'көп': ['к', 'ө', 'п'],
 ',': [','],
 'көк': ['к', 'ө', 'к'],
 '.': ['.'],
 'үш': ['ү', 'ш'],
 'қыз': ['қы', 'з'],
 'да': ['д', 'а'],
 'ұшқыш': ['ұ', 'ш', 'қы', 'ш'],
 'та': ['т', 'а'],
 '–': ['–'],
 'шын': ['ш', 'ын'],
 'айтты': ['а', 'й', 'т', 'ты'],
 'ма': ['м', 'а'],
 'сын': ['сын'],
 'айтса': ['а', 'й', 'т', 'са'],
 'шыны': ['ш', 'ын', 'ы'],
 'сыны': ['сын', 'ы'],
 'болсын': ['б', 'о', 'л', 'сын'],
 '!': ['!']}

In [296]:
merges

{('т', 'ы'): 'ты',
 ('ы', 'н'): 'ын',
 ('с', 'а'): 'са',
 ('а', 'л'): 'ал',
 ('ал', 'д'): 'алд',
 ('алд', 'ы'): 'алды',
 ('қ', 'ы'): 'қы',
 ('са', 'ты'): 'саты',
 ('к', 'е'): 'ке',
 ('ш', 'е'): 'ше',
 ('с', 'ын'): 'сын',
 ('саты', 'б'): 'сатыб',
 ('сатыб', 'алды'): 'сатыбалды',
 ('ке', 'ше'): 'кеше',
 ('п', 'т'): 'пт',
 ('пт', 'е'): 'пте',
 ('пте', 'р'): 'птер',
 ('ү', 'ш'): 'үш',
 ('қы', 'з'): 'қыз',
 ('ұ', 'ш'): 'ұш',
 ('ұш', 'қы'): 'ұшқы',
 ('ұшқы', 'ш'): 'ұшқыш',
 ('ш', 'ын'): 'шын',
 ('а', 'й'): 'ай',
 ('ай', 'т'): 'айт',
 ('д', 'а'): 'да',
 ('н', 'е'): 'не',
 ('не', 'ше'): 'неше',
 ('д', 'ә'): 'дә',
 ('дә', 'птер'): 'дәптер',
 ('саты', 'п'): 'сатып',
 ('ке', 'птер'): 'кептер',
 ('а', 'ты'): 'аты',
 ('аты', 'п'): 'атып',
 ('к', 'ө'): 'кө',
 ('айт', 'ты'): 'айтты',
 ('м', 'а'): 'ма',
 ('айт', 'са'): 'айтса',
 ('кө', 'п'): 'көп',
 ('кө', 'к'): 'көк',
 ('т', 'а'): 'та',
 ('шын', 'ы'): 'шыны',
 ('сын', 'ы'): 'сыны',
 ('б', 'о'): 'бо',
 ('бо', 'л'): 'бол'}

In [51]:
def tokenize(text):
    pre_tokenize_result = tokenizer._tokenizer.pre_tokenizer.pre_tokenize_str(text)
    pre_tokenized_text = [word for word, offset in pre_tokenize_result]
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

In [52]:
splits

{'сатыбалды': ['саты', 'б', 'алды'],
 'кеше': ['ке', 'ше'],
 'неше': ['н', 'е', 'ше'],
 'дәптер': ['д', 'ә', 'п', 'т', 'е', 'р'],
 'сатып': ['саты', 'п'],
 'алды': ['алды'],
 '?': ['?'],
 'кептер': ['ке', 'п', 'т', 'е', 'р'],
 'атып': ['а', 'ты', 'п'],
 'көп': ['к', 'ө', 'п'],
 ',': [','],
 'көк': ['к', 'ө', 'к'],
 '.': ['.'],
 'үш': ['ү', 'ш'],
 'қыз': ['қы', 'з'],
 'да': ['д', 'а'],
 'ұшқыш': ['ұ', 'ш', 'қы', 'ш'],
 'та': ['т', 'а'],
 '–': ['–'],
 'шын': ['ш', 'ын'],
 'айтты': ['а', 'й', 'т', 'ты'],
 'ма': ['м', 'а'],
 'сын': ['с', 'ын'],
 'айтса': ['а', 'й', 'т', 'са'],
 'шыны': ['ш', 'ын', 'ы'],
 'сыны': ['с', 'ын', 'ы'],
 'болсын': ['б', 'о', 'л', 'с', 'ын'],
 '!': ['!']}

In [291]:
for i in corpus:
    print(tokenize(i))

['сатыбалды', 'кеше', 'неше', 'дәптер', 'сатып', 'алды', '?']
['сатыбалды', 'кеше', 'неше', 'кептер', 'атып', 'алды', '?']
['сатыбалды', 'кеше', 'көп', 'дәптер', 'сатып', 'алды', ',']
['сатыбалды', 'кеше', 'көк', 'кептер', 'атып', 'алды', '.']
['үш', 'қыз', 'да', 'ұшқыш', ',']
['ұшқыш', 'та', 'үш', 'қыз', '.']
['үш', 'қыз', '–', 'ұшқыш', ',']
['ұшқыш', '–', 'үш', 'қыз', '.']
['шын', 'айтты', 'ма', '?']
['сын', 'айтты', 'ма', '?']
['шын', 'айтса', 'да', ',']
['сын', 'айтса', 'да', ',']
['шыны', '–', 'сын', ',']
['сыны', '–', 'шын', 'бол', 'сын', '!']
